In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
%cd /content/drive/My Drive/ML Project

/content/drive/My Drive/ML Project


# **Setup Git: [Repo](https://github.com/Gabin-ishimwe/auto-scaling-prediction)**
**⚠️ Don't run these cells**

In [ ]:
!git version
!git help

In [ ]:
!ls

logfiles.log  logfiles.log.zip	ML_codebase.ipynb  test_generator.ipynb


In [ ]:
!echo "# auto-scaling-prediction" >> README.md

In [ ]:
!echo "logfiles.log" >> .gitignore

In [ ]:
!git init
!git add .
!git commit -m "first commit"
!git branch -M main
!git push -u origin main

In [6]:
!git config --global user.email "g.ishimwe@alustudent.com"
!git config --global user.name "Gabin-ishimwe"

In [ ]:
!git commit -m "initial commit"

In [ ]:
!git status

# **Data Preprocessing**

In [ ]:
import numpy as np
import pandas as pd
import re # regex expression modules
import os # os modules
import time # time modules
from tqdm import tqdm # Instantly make your loops show a smart progress meter

In [ ]:
common_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-)'
combined_regex = '^(?P<client>\S+) \S+ (?P<userid>\S+) \[(?P<datetime>[^\]]+)\] "(?P<method>[A-Z]+) (?P<request>[^ "]+)? HTTP/[0-9.]+" (?P<status>[0-9]{3}) (?P<size>[0-9]+|-) "(?P<referrer>[^"]*)" "(?P<useragent>[^"]*)'
columns = ['client', 'userid', 'datetime', 'method', 'request', 'status', 'size', 'referer', 'user_agent']
def logs_to_df(logfile, output_dir, errors_file):
    with open(logfile) as source_file:
        linenumber = 0
        parsed_lines = []
        for line in tqdm(source_file):
            try:
                log_line = re.findall(combined_regex, line)[0]
                parsed_lines.append(log_line)
            except Exception as e:
                with open(errors_file, 'at') as errfile:
                    print((line, str(e)), file=errfile)
                continue
            linenumber += 1
            if linenumber % 250_000 == 0:
                df = pd.DataFrame(parsed_lines, columns=columns)
                df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
                parsed_lines.clear()
        else:
            df = pd.DataFrame(parsed_lines, columns=columns)
            df.to_parquet(f'{output_dir}/file_{linenumber}.parquet')
            parsed_lines.clear()

In [ ]:
!git status

On branch main
Your branch is ahead of 'origin/main' by 3 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [ ]:
!git add .
!git commit -m "ft: hide secrets"

[main 1346774] ft: hide secrets
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:
!git log --oneline --graph

In [3]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [4]:
!git branch